In [140]:
import os
print(os.getcwd())

/Users/pking/Downloads


In [92]:
import urllib.request
with urllib.request.urlopen('https://docs.splunk.com/Documentation/Splunk/8.0.0/Security/UseAuthTokens') as response:
   html = response.read()

In [93]:
print(html)

b'<!DOCTYPE html>\n<!--[if lt IE 7]><html class="no-js lt-ie9 lt-ie8 lt-ie7"><![endif]-->\n<!--[if IE 7]><html class="no-js lt-ie9 lt-ie8"><![endif]-->\n<!--[if IE 8]><html class="no-js lt-ie9"><![endif]-->\n<!--[if gt IE 8]><!--><html class="client-nojs" lang="en" dir="ltr"><!--<![endif]-->\n<head>\n<meta charset="UTF-8"/>\n<title>Use authentication tokens - Splunk Documentation</title>\n<script>document.documentElement.className = document.documentElement.className.replace( /(^|\\s)client-nojs(\\s|$)/, "$1client-js$2" );</script>\n<script>(window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"Documentation","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":100,"wgPageName":"Documentation:Splunk:Security:UseAuthTokens:8.0.0","wgTitle":"Splunk:Security:UseAuthTokens:8.0.0","wgCurRevisionId":1056402,"wgRevisionId":1056402,"wgArticleId":186161,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":[

In [100]:
from bs4 import BeautifulSoup

url = 'https://docs.splunk.com/Documentation:TA-ueba:User:Sourcetypes:1.2.0'#'https://docs.splunk.com/Documentation:Splunkbase:Splunkbase:Versioncompatibility:splunkbase'

conn = urllib.request.urlopen(url)
html = conn.read()

soup = BeautifulSoup(html)

links = soup.find_all('a')

for tag in links:
    link = tag.get('href', None)
    if link is not None:
        print(link)

/Documentation
http://www.splunk.com/en_us/products.html
http://www.splunk.com/en_us/products.html
http://www.splunk.com/en_us/products/splunk-enterprise.html
http://www.splunk.com/en_us/products/splunk-cloud.html
http://www.splunk.com/en_us/products/splunk-enterprise.html
http://www.splunk.com/en_us/products/premium-solutions/splunk-enterprise-security.html
http://www.splunk.com/en_us/products/premium-solutions/it-service-intelligence.html
http://www.splunk.com/en_us/products/premium-solutions/user-behavior-analytics.html
http://www.splunk.com/en_us/products/apps-and-add-ons.html
http://www.splunk.com/en_us/products/pricing.html
http://www.co-store.com/splunk
http://www.splunk.com/en_us/solutions
http://www.splunk.com/en_us/solutions/solution-areas.html
http://www.splunk.com/en_us/solutions/solution-areas/application-delivery.html
http://www.splunk.com/en_us/solutions/solution-areas/big-data.html
http://www.splunk.com/en_us/solutions/solution-areas/business-analytics.html
http://www.s

In [135]:
import csv

out_file = open('out_file.csv', mode='w')
out_writer = csv.writer(out_file, delimiter=',')
out_writer.writerow(['Topic', 'Status'])

14

In [138]:
def testlink(link, page, out_writer):
    try:
        test = urllib.request.urlopen(link)
    except urllib.error.HTTPError as e:
        print('    HTTPError: {}'.format(e.code))
        out_writer.writerow([page, '404 (' + link + ')'])
    else:
        if test.geturl() == 'https://dev.splunk.com/' and \
            link != 'https://dev.splunk.com/' and \
            link != 'https://dev.splunk.com' and \
            link != 'http://dev.splunk.com/' and \
            link != 'http://dev.splunk.com':
                print('    Not mapped: ' + link)
                out_writer.writerow([page, 'Not mapped (' + link + ')'])
        else: 
            out_writer.writerow([page, 'Ok (' + link + ')'])

In [141]:
def getlinks(page, out_writer):
    try:
        conn = urllib.request.urlopen(page)
        html = conn.read()
        soup = BeautifulSoup(html)
        links = soup.find_all('a')
        for tag in links:
            link = tag.get('href', None)
            if link is not None:
                if link.find('dev.splunk.com') > -1: 
                    testlink(link, page, out_writer)
    except urllib.error.HTTPError as e:
        print('    PonyDocs HTTPError: {}'.format(e.code))    

In [143]:
with open('Dev links from Ponydocs - Sheet1.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            print('Row ' + str(line_count) + ' Page: ' + ponypage)
            ponypage = row[0] #print(row[0]) # print(row)
            getlinks(ponypage, out_writer)
            line_count += 1

Row 1 Page: https://docs.splunk.com/Documentation:ES:Admin:IRoverview:5.2.0
Row 2 Page: https://docs.splunk.com/Documentation:AddOns:Akamai:Configuresharedtokens:previous
Row 3 Page: https://docs.splunk.com/Documentation:AddOns:Akamai:Configuresharedtokens:unreleased
Row 4 Page: https://docs.splunk.com/Documentation:AddOns:Akamai:Hardwareandsoftwarerequirements:previous
Row 5 Page: https://docs.splunk.com/Documentation:AddOns:Akamai:Hardwareandsoftwarerequirements:unreleased
Row 6 Page: https://docs.splunk.com/Documentation:AddOns:Akamai:Install:previous
Row 7 Page: https://docs.splunk.com/Documentation:AddOns:Akamai:Install:unreleased
Row 8 Page: https://docs.splunk.com/Documentation:AddOns:Firehose:ConfigureHECdistributed:previous
    HTTPError: 404
Row 9 Page: https://docs.splunk.com/Documentation:AddOns:Firehose:ConfigureHECdistributed:released
Row 10 Page: https://docs.splunk.com/Documentation:AddOns:Firehose:ConfigureHECdistributed:staging
Row 11 Page: https://docs.splunk.com/Doc

Row 93 Page: https://docs.splunk.com/Documentation:ES:RN:Enhancements:Dino
Row 94 Page: https://docs.splunk.com/Documentation:ES:User:Audit:6.0.0
Row 95 Page: https://docs.splunk.com/Documentation:ES:User:Audit:F1
Row 96 Page: https://docs.splunk.com/Documentation:ES:User:Audit:Interceptor
Row 97 Page: https://docs.splunk.com/Documentation:ES:User:Audit:drafts
Row 98 Page: https://docs.splunk.com/Documentation:ES:User:Configureblocklists:Dino
Row 99 Page: https://docs.splunk.com/Documentation:ES:User:NotableEvents:4.5.0
Row 100 Page: https://docs.splunk.com/Documentation:ES:User:RiskScoring:4.7.0
Row 101 Page: https://docs.splunk.com/Documentation:ES:User:RiskScoring:drafts
Row 102 Page: https://docs.splunk.com/Documentation:ES:User:SetupAdaptiveResponse:4.1.0
Row 103 Page: https://docs.splunk.com/Documentation:ES:dev:Planningyourintegration:1.1
Row 104 Page: https://docs.splunk.com/Documentation:Hunk:Hunk:Indexes:Sandbox
Row 105 Page: https://docs.splunk.com/Documentation:Hunk:Hunk:Pr

Row 184 Page: https://docs.splunk.com/Documentation:Splunk:Data:UsetheHTTPEventCollector:7.1.0
Row 185 Page: https://docs.splunk.com/Documentation:Splunk:DataOB:BestpracticesforindexingdatawithHEC:1
Row 186 Page: https://docs.splunk.com/Documentation:Splunk:DataOB:Informationonotherdataon-boardingtools:1
    HTTPError: 404
Row 187 Page: https://docs.splunk.com/Documentation:Splunk:Deploy:Postdeployment:6.5.0
Row 188 Page: https://docs.splunk.com/Documentation:Splunk:Developer:DevIntro:4.3
Row 189 Page: https://docs.splunk.com/Documentation:Splunk:Developer:ExtendSplunkIntro:4.2beta
Row 190 Page: https://docs.splunk.com/Documentation:Splunk:Developer:GetStarted:3.3
Row 191 Page: https://docs.splunk.com/Documentation:Splunk:Developer:HowToUseThisManual:4.3
Row 192 Page: https://docs.splunk.com/Documentation:Splunk:Developer:OutputFormatsViaXML:3.3
Row 193 Page: https://docs.splunk.com/Documentation:Splunk:Developer:Properties:3.2
Row 194 Page: https://docs.splunk.com/Documentation:Splunk

Row 274 Page: https://docs.splunk.com/Documentation:Splunk:RESTREF:RESTindexExamples:6.2beta
Row 275 Page: https://docs.splunk.com/Documentation:Splunk:RESTREF:RESTinput:8.0.0
Row 276 Page: https://docs.splunk.com/Documentation:Splunk:RESTREF:RESTinputExamples:6.5.0
Row 277 Page: https://docs.splunk.com/Documentation:Splunk:RESTREF:RESTintrospect:8.0.0
Row 278 Page: https://docs.splunk.com/Documentation:Splunk:RESTREF:RESTintrospectExamples:6.3.0beta
Row 279 Page: https://docs.splunk.com/Documentation:Splunk:RESTREF:RESTknowledge:8.0.0
Row 280 Page: https://docs.splunk.com/Documentation:Splunk:RESTREF:RESTknowledgeExamples:6.2beta
Row 281 Page: https://docs.splunk.com/Documentation:Splunk:RESTREF:RESTkvstore:7.1.0
Row 282 Page: https://docs.splunk.com/Documentation:Splunk:RESTREF:RESTlicenseExamples:6.3.0beta
Row 283 Page: https://docs.splunk.com/Documentation:Splunk:RESTREF:RESTmetrics:7.3.1
Row 284 Page: https://docs.splunk.com/Documentation:Splunk:RESTREF:RESTsearch:8.0.0
Row 285 Pa